In [8]:
from sympy import *
from sympy.parsing.sympy_parser import parse_expr
from sympy.printing.latex import LatexPrinter
from sympy.solvers.solveset import linsolve
from sympy.solvers import solve_poly_system
from IPython.display import display, Latex
import numpy as np

In [9]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [12]:
def steepest_descent(str_expr, str_symbols, x_init, tau = None, n_iter=10, rounding_digit=4):
    sym = symbols(str_symbols)
    f = Function('f')(*sym)
    
    expr = parse_expr(str_expr)
    print('The function is')
    display(Eq(f,expr))
    grad = MatrixSymbol('\u2207f',len(sym),1)
    print('\nStep-1: Finding gradient and hessian Matrix of the function')
    display(Eq(grad, Matrix([Derivative(f,x) for x in sym])))
    display(Eq(grad, Matrix([Derivative(expr,x) for x in sym])))
    grad_m = Matrix([diff(expr,x) for x in sym])
    display(Eq(grad, grad_m))
    
    print('\nCreating the Hessian Matrix')
    hess = MatrixSymbol('Hf',len(sym),len(sym))
    hess_m_f = zeros(len(sym))
    hess_m_expr = zeros(len(sym))
    hess_m = zeros(len(sym))
    for i,x in enumerate(sym):
        for j,y in enumerate(sym):
            hess_m_f[i,j] = Derivative(Derivative(f,y),x)
            hess_m_expr[i,j] = Derivative(Derivative(expr,y),x)
            hess_m[i,j] = diff(diff(expr,y),x)
    display(Eq(hess,hess_m_f))  
    display(Eq(hess,hess_m_expr)) 
    display(Eq(hess,hess_m))
    print('\nStep-2: Computing the step size \u03C4\n')
    #display(hess_m.subs(sym[1], Symbol('y_i')))
   
    sym_list = str_symbols.split()#[x.strip() for x in  list(''.join(str_symbols)) if x !=' ']
    grad_m_i = grad_m.subs([(s,Symbol(f'{t}_i')) for s,t in zip(sym,sym_list)])
    hess_m_i = hess_m.subs([(s,Symbol(f'{t}_i')) for s,t in zip(sym,sym_list)])
    tau_sym = Symbol('\u03C4_i')
    grad_i = Symbol('\u2207f_i')
    grad_i_T = Symbol('\u2207f_i^T')
    hess_i = Symbol('Hf_i')
    
    if not tau:
        tau_f = Mul(Mul(grad_i_T,grad_i), Pow(Mul(Mul(grad_i_T,hess_i),grad_i), Integer(-1)), evaluate=False)

    #     Mul(Mul(Mul(Mul(Transpose(grad_i),grad_i),Pow(Transpose(grad_i),Integer(-1))),Pow(hess_i, Integer(-1))),
    #                 Pow(grad_i, Integer(-1)))

        #display(Eq(tau,tau_f))
        print('\u03C4\u1D62 = \u2207f\u1D62\u1D40*\u2207f\u1D62/\u2207f\u1D62\u1D40*Hf\u1D62*\u2207f\u1D62\n')
        tau_expr = MatMul(MatMul(Transpose(grad_m_i),grad_m_i), Pow(MatMul(MatMul(Transpose(grad_m_i),hess_m_i, evaluate=False)
                                                                  ,grad_m_i, evaluate=False)
                                                              , Integer(-1)), evaluate=False)
        display(tau_expr)
        num = (Transpose(grad_m_i)*grad_m_i)[0]
        den = (Transpose(grad_m_i)*hess_m_i*grad_m_i)[0]
        tau_expr = num/den
    else:
        tau_expr = tau
        
    display(Eq(tau_sym,tau_expr))
    
    print('\nStep-3: Iterate the minimum point')
    display(Eq(Symbol('x_i_+_1'), Symbol('x_i') - tau_sym*grad_i))
    sym_i = [Symbol(f'{t}_i') for t in sym_list]
    sym_i_plus = [Symbol(f'{t}_i_+_1') for t in sym_list]
    display(Eq(Matrix(sym_i_plus), MatAdd(Matrix(sym_i), - MatMul(tau_sym,grad_m_i), evaluate=False)))
    display(Eq(Matrix(sym_i_plus), MatAdd(Matrix(sym_i), - MatMul(tau_expr,grad_m_i))))
    
    x_old = x_init
    display(Eq(Matrix([Symbol(f'{t}_0') for t in sym_list]), Matrix(x_old)))
    for i in range(n_iter):
        print(f'\nIteration - {i+1}:\n')
        sym_old = [Symbol(f'{t}_{i}') for t in sym_list]
        sym_new = [Symbol(f'{t}_{i+1}') for t in sym_list]
        display(Eq(Symbol(f'tau_{i}'),  tau_expr if isinstance(tau_expr, float) else 
                   tau_expr.subs([(s,t) for s,t in zip(sym_i, sym_old)])))
                
        display(Eq(Symbol(f'tau_{i}'),  tau_expr if isinstance(tau_expr, float) else
                   tau_expr.subs([(s,t) for s,t in zip(sym_i, x_old)])))
                
        display(Eq(Matrix(sym_new), MatAdd(Matrix(sym_old), - MatMul(tau_expr if isinstance(tau_expr, float) else
                                                    tau_expr.subs([(s,t) for s,t in zip(sym_i, sym_old)])
                                                                     ,grad_m_i.subs([(s,t) for s,t in zip(sym_i, sym_old)])))))
        display(Eq(Matrix(sym_new), MatAdd(Matrix(x_old), - MatMul(tau_expr if isinstance(tau_expr, float) else
                                                                tau_expr.subs([(s,t) for s,t in zip(sym_i, x_old)])
                                                                     ,grad_m_i.subs([(s,t) for s,t in zip(sym_i, x_old)]),
                                                                  evaluate=False))))
        x_new = Add(Matrix(x_old), - MatMul(tau_expr if isinstance(tau_expr, float) else
                                            tau_expr.subs([(s,t) for s,t in zip(sym_i, x_old)])
                                                                     ,grad_m_i.subs([(s,t) for s,t in zip(sym_i, x_old)]),
                                                                  evaluate=False))
        x_new = [N(element, rounding_digit) for element in x_new]
        if i ==0:
            display(Eq(Matrix(sym_new), Add(Matrix(x_old), - MatMul(tau_expr if isinstance(tau_expr, float) else
                                                                    tau_expr.subs([(s,t) for s,t in zip(sym_i, x_old)])
                                                                         ,grad_m_i.subs([(s,t) for s,t in zip(sym_i, x_old)]),
                                                                      evaluate=False))))
        display(Eq(Matrix(sym_new), Matrix(x_new)))
        
        display(Eq(f.subs([(s,t) for s,t in zip(sym, sym_new)]), expr.subs([(s,t) for s,t in zip(sym, sym_new)])))
        
        display(Eq(f.subs([(s,t) for s,t in zip(sym, x_new)]), expr.subs([(s,t) for s,t in zip(sym, x_new)])))
        
        x_old = x_new
        
        #display(Eq(Matrix(sym_i_plus), MatAdd(Matrix(sym_i), - MatMul(tau_expr,grad_m_i))))

In [6]:
#variable tau
steepest_descent(str_expr='x**2 - 2*y**2 + 4*z**2' , str_symbols= 'x y z', x_init=[-1, 2, 0], 
                 tau=None, n_iter=10, rounding_digit = 4)

The function is


Eq(f(x, y, z), x**2 - 2*y**2 + 4*z**2)


Step-1: Finding gradient and hessian Matrix of the function


Eq(∇f, Matrix([
[Derivative(f(x, y, z), x)],
[Derivative(f(x, y, z), y)],
[Derivative(f(x, y, z), z)]]))

Eq(∇f, Matrix([
[Derivative(x**2 - 2*y**2 + 4*z**2, x)],
[Derivative(x**2 - 2*y**2 + 4*z**2, y)],
[Derivative(x**2 - 2*y**2 + 4*z**2, z)]]))

...

Eq(Matrix([
[x_10],
[y_10],
[z_10]]), (-(4*x_9**2 + 16*y_9**2 + 64*z_9**2)/(8*x_9**2 - 64*y_9**2 + 512*z_9**2))*Matrix([
[ 2*x_9],
[-4*y_9],
[ 8*z_9]]) + Matrix([
[x_9],
[y_9],
[z_9]]))

Eq(Matrix([
[x_10],
[y_10],
[z_10]]), Matrix([
[ -0.01878],
[-0.002347],
[        0]]) + Matrix([
[   0.0228],
[-0.005701],
[        0]]))

Eq(Matrix([
[x_10],
[y_10],
[z_10]]), Matrix([
[ 0.004025],
[-0.008049],
[        0]]))

Eq(f(x_10, y_10, z_10), x_10**2 - 2*y_10**2 + 4*z_10**2)

Eq(f(0.004025, -0.008049, 0), -0.0001134)

In [13]:
#constant tau
steepest_descent(str_expr='3*x**2 + 4*y**2 + 7*x' , str_symbols= 'x y', x_init=[4,3], 
                 tau=0.1, n_iter=10, rounding_digit = 4)

The function is


Eq(f(x, y), 3*x**2 + 7*x + 4*y**2)


Step-1: Finding gradient and hessian Matrix of the function


Eq(∇f, Matrix([
[Derivative(f(x, y), x)],
[Derivative(f(x, y), y)]]))

Eq(∇f, Matrix([
[Derivative(3*x**2 + 7*x + 4*y**2, x)],
[Derivative(3*x**2 + 7*x + 4*y**2, y)]]))

...

Eq(Matrix([
[x_10],
[y_10]]), Matrix([
[x_9],
[y_9]]) + Matrix([
[-0.6*x_9 - 0.7],
[      -0.8*y_9]]))

Eq(Matrix([
[x_10],
[y_10]]), Matrix([
[  -1.165],
[1.536e-6]]) + Matrix([
[  -0.000811767578125],
[-1.22878700494766e-6]]))

Eq(Matrix([
[x_10],
[y_10]]), Matrix([
[  -1.166],
[3.072e-7]]))

Eq(f(x_10, y_10), 3*x_10**2 + 7*x_10 + 4*y_10**2)

Eq(f(-1.166, 3.072e-7), -4.083)